## Analyze Datasets for Joins

Let us analyze both January 2008 airtraffic as well as airport codes data sets that are going to be used for joins.

In [1]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/jEKSFZLWiEo?rel=0&amp;controls=1&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>

* We will use January 2008 airtraffic data which have all relevant flight details such as departure, arrival etc.
* As part of the analysis, we also might want additional metadata such as city, state etc for airports. We got that information as part of airport codes data set.
* Let us read and review both January 2008 airtraffic data as well as airport codes data set.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [1]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Joining Data Sets'). \
    master('yarn'). \
    getOrCreate()

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

In [2]:
spark.conf.set("spark.sql.shuffle.partitions", "2")

In [3]:
airtraffic = spark. \
    read. \
    parquet("/public/airtraffic_all/airtraffic-part/flightmonth=200801")

In [4]:
airtraffic.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Car

In [5]:
airtraffic.show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|IsArrDelayed|IsDepDelayed|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+------------+------------+
|2008|    1|    

* We will be using another data set to get details about airports. Details include information such as State, City etc for a given airport code.
* Let us analyze the Dataset to confirm if there is header and also how the data is structured.

In [6]:
airportCodesPath = "/public/airtraffic_all/airport-codes"

In [7]:
spark. \
    read. \
    text(airportCodesPath). \
    show(truncate=False)

+-------------------------+
|value                    |
+-------------------------+
|City	State	Country	IATA  |
|Abbotsford	BC	Canada	YXX |
|Aberdeen	SD	USA	ABR      |
|Abilene	TX	USA	ABI       |
|Akron	OH	USA	CAK         |
|Alamosa	CO	USA	ALS       |
|Albany	GA	USA	ABY        |
|Albany	NY	USA	ALB        |
|Albuquerque	NM	USA	ABQ   |
|Alexandria	LA	USA	AEX    |
|Allentown	PA	USA	ABE     |
|Alliance	NE	USA	AIA      |
|Alpena	MI	USA	APN        |
|Altoona	PA	USA	AOO       |
|Amarillo	TX	USA	AMA      |
|Anahim Lake	BC	Canada	YAA|
|Anchorage	AK	USA	ANC     |
|Appleton	WI	USA	ATW      |
|Arviat	NWT	Canada	YEK    |
|Asheville	NC	USA	AVL     |
+-------------------------+
only showing top 20 rows



 * Data is tab separated.
 * There is header for the data set.
 * Dataset have 4 fields - **Country, State, City, IATA**
    
    
Create DataFrame airport_codes applying appropriate Schema.


In [8]:
airportCodesPath = "/public/airtraffic_all/airport-codes"

In [9]:
airportCodes = spark. \
    read. \
    option("sep", "\t"). \
    option("header", True). \
    option("inferSchema", True). \
    csv(airportCodesPath)

* Preview and Understand the data.

In [10]:
airportCodes.show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
|   Alliance|   NE|    USA| AIA|
|     Alpena|   MI|    USA| APN|
|    Altoona|   PA|    USA| AOO|
|   Amarillo|   TX|    USA| AMA|
|Anahim Lake|   BC| Canada| YAA|
|  Anchorage|   AK|    USA| ANC|
|   Appleton|   WI|    USA| ATW|
|     Arviat|  NWT| Canada| YEK|
|  Asheville|   NC|    USA| AVL|
|      Aspen|   CO|    USA| ASE|
+-----------+-----+-------+----+
only showing top 20 rows



* Get schema of **airport_codes**.

In [11]:
airportCodes.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- IATA: string (nullable = true)



* Get the count of records

In [12]:
airportCodes.count()

526

   * Get the count of unique records and see if it is the same as total count.

In [13]:
airportCodes. \
    select("IATA"). \
    distinct(). \
    count()

524

 * If they are not equal, analyze the data and identify IATA codes which are repeated more than once.

In [14]:
from pyspark.sql.functions import lit, count

In [15]:
duplicateIATACount = airportCodes. \
    groupBy("IATA"). \
    agg(count(lit(1)).alias("iata_count")). \
    filter("iata_count > 1")

In [16]:
duplicateIATACount.show()

+----+----------+
|IATA|iata_count|
+----+----------+
| Big|         3|
+----+----------+



 * Filter out the duplicates using the most appropriate one and discard others.

In [17]:
airportCodes. \
    filter("IATA = 'Big'"). \
    show()

+-----------+------+-------+----+
|       City| State|Country|IATA|
+-----------+------+-------+----+
|       Hilo|    HI|    USA| Big|
|Kailua-Kona|Hawaii|    USA| Big|
|    Kamuela|Hawaii|    USA| Big|
+-----------+------+-------+----+



In [18]:
airportCodes. \
    filter("!(State = 'Hawaii' AND IATA = 'Big')"). \
    show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
|   Alliance|   NE|    USA| AIA|
|     Alpena|   MI|    USA| APN|
|    Altoona|   PA|    USA| AOO|
|   Amarillo|   TX|    USA| AMA|
|Anahim Lake|   BC| Canada| YAA|
|  Anchorage|   AK|    USA| ANC|
|   Appleton|   WI|    USA| ATW|
|     Arviat|  NWT| Canada| YEK|
|  Asheville|   NC|    USA| AVL|
|      Aspen|   CO|    USA| ASE|
+-----------+-----+-------+----+
only showing top 20 rows



In [19]:
airportCodes. \
    filter("!(State = 'Hawaii' AND IATA = 'Big')"). \
    count()

524

 * Get number of airports (IATA Codes) for each state in the US. Sort the data in descending order by count.

In [20]:
airportCodesPath = "/public/airtraffic_all/airport-codes"

In [21]:
airportCodes = spark. \
    read. \
    option("sep", "\t"). \
    option("header", True). \
    option("inferSchema", True). \
    csv(airportCodesPath). \
    filter("!(State = 'Hawaii' AND IATA = 'Big') AND Country = 'USA'")

In [22]:
airportCodes.count()

443

In [23]:
from pyspark.sql.functions import count, col, lit

In [24]:
airportCountByState = airportCodes. \
    groupBy("Country", "State"). \
    agg(count(lit(1)).alias("IATACount")). \
    orderBy(col("IATACount").desc())

In [25]:
airportCountByState.show(51)

+-------+-----+---------+
|Country|State|IATACount|
+-------+-----+---------+
|    USA|   CA|       29|
|    USA|   TX|       26|
|    USA|   AK|       25|
|    USA|   NY|       18|
|    USA|   MI|       18|
|    USA|   FL|       18|
|    USA|   MT|       14|
|    USA|   PA|       13|
|    USA|   IL|       12|
|    USA|   CO|       12|
|    USA|   WY|       10|
|    USA|   NC|       10|
|    USA|   WI|        9|
|    USA|   NE|        9|
|    USA|   GA|        9|
|    USA|   NM|        9|
|    USA|   HI|        9|
|    USA|   WA|        9|
|    USA|   KS|        9|
|    USA|   ND|        8|
|    USA|   MO|        8|
|    USA|   AR|        8|
|    USA|   MA|        8|
|    USA|   MN|        8|
|    USA|   AZ|        8|
|    USA|   WV|        8|
|    USA|   IA|        8|
|    USA|   SD|        7|
|    USA|   ME|        7|
|    USA|   VA|        7|
|    USA|   LA|        7|
|    USA|   MS|        7|
|    USA|   OR|        7|
|    USA|   TN|        6|
|    USA|   AL|        6|
|    USA|   

In [26]:
airportCountByState.count()

51